In [8]:
import argparse

# 创建命令行参数解析器
parser = argparse.ArgumentParser(description='PyTorch video prediction model - PredRNN')

# 添加训练/测试相关的参数
parser.add_argument('--is_training', type=int, default=1)
parser.add_argument('--device', type=str, default='cuda')

# 添加数据集相关的参数
parser.add_argument('--dataset_name', type=str, default='swan')
parser.add_argument('--train_data_paths', type=str, default='E:/Dataset/met_waves/Swan4predRNN/train.npy')
parser.add_argument('--valid_data_paths', type=str, default='E:/Dataset/met_waves/Swan4predRNN/val.npy')
parser.add_argument('--save_dir', type=str, default='checkpoints/swan_predrnn_v2')
parser.add_argument('--gen_frm_dir', type=str, default='results/swan_predrnn_v2')
parser.add_argument('--input_length', type=int, default=10)
parser.add_argument('--total_length', type=int, default=20)
parser.add_argument('--img_width', type=int, default=64)
parser.add_argument('--img_channel', type=int, default=1)

# 添加模型相关的参数
parser.add_argument('--model_name', type=str, default='predrnn_v2')
parser.add_argument('--pretrained_model', type=str, default='')
parser.add_argument('--num_hidden', type=str, default='128,128,128,128')
parser.add_argument('--filter_size', type=int, default=5)
parser.add_argument('--stride', type=int, default=1)
parser.add_argument('--patch_size', type=int, default=4)
parser.add_argument('--layer_norm', type=int, default=0)
parser.add_argument('--decouple_beta', type=float, default=0.1)

# 添加逆向调度采样相关的参数
parser.add_argument('--reverse_scheduled_sampling', type=int, default=1)
parser.add_argument('--r_sampling_step_1', type=float, default=25000)
parser.add_argument('--r_sampling_step_2', type=int, default=50000)
parser.add_argument('--r_exp_alpha', type=int, default=2500)
# 添加调度采样相关的参数
parser.add_argument('--scheduled_sampling', type=int, default=1)
parser.add_argument('--sampling_stop_iter', type=int, default=50000)
parser.add_argument('--sampling_start_value', type=float, default=1.0)
parser.add_argument('--sampling_changing_rate', type=float, default=0.00002)

# 添加优化相关的参数
parser.add_argument('--lr', type=float, default=0.0001)
parser.add_argument('--reverse_input', type=int, default=1)
parser.add_argument('--batch_size', type=int, default=8)
parser.add_argument('--max_iterations', type=int, default=80000)
parser.add_argument('--display_interval', type=int, default=100)
parser.add_argument('--test_interval', type=int, default=5000)
parser.add_argument('--snapshot_interval', type=int, default=5000)
parser.add_argument('--num_save_samples', type=int, default=10)
parser.add_argument('--n_gpu', type=int, default=1)

# 添加可视化相关的参数
parser.add_argument('--visual', type=int, default=0)
parser.add_argument('--visual_path', type=str, default='./decoupling_visual')

# 添加基于动作的PredRNN相关的参数
parser.add_argument('--injection_action', type=str, default='concat')
parser.add_argument('--conv_on_input', type=int, default=0, help='conv on input')
parser.add_argument('--res_on_conv', type=int, default=0, help='res on conv')
parser.add_argument('--num_action_ch', type=int, default=4, help='num action ch')
# 解析命令行参数
args, unknown = parser.parse_known_args()
# 打印解析后的参数
print(args)

Namespace(batch_size=8, conv_on_input=0, dataset_name='swan', decouple_beta=0.1, device='cuda', display_interval=100, filter_size=5, gen_frm_dir='results/swan_predrnn_v2', img_channel=1, img_width=64, injection_action='concat', input_length=10, is_training=1, layer_norm=0, lr=0.0001, max_iterations=80000, model_name='predrnn_v2', n_gpu=1, num_action_ch=4, num_hidden='128,128,128,128', num_save_samples=10, patch_size=4, pretrained_model='', r_exp_alpha=2500, r_sampling_step_1=25000, r_sampling_step_2=50000, res_on_conv=0, reverse_input=1, reverse_scheduled_sampling=1, sampling_changing_rate=2e-05, sampling_start_value=1.0, sampling_stop_iter=50000, save_dir='checkpoints/swan_predrnn_v2', scheduled_sampling=1, snapshot_interval=5000, stride=1, test_interval=5000, total_length=20, train_data_paths='E:/Dataset/met_waves/Swan4predRNN/train.npy', valid_data_paths='E:/Dataset/met_waves/Swan4predRNN/val.npy', visual=0, visual_path='./decoupling_visual')


In [2]:
import math
import numpy as np


def reserve_schedule_sampling_exp(itr):
    """
    根据当前迭代次数计算逆向调度采样的概率，并生成相应的采样标志。

    参数:
    itr (int): 当前迭代次数

    返回:
    real_input_flag (np.ndarray): 采样标志数组
    """
    # 根据当前迭代次数计算逆向调度采样的概率
    #r_eta表示逆向调度采样的概率值
    if itr < args.r_sampling_step_1:
        r_eta = 0.5
    elif itr < args.r_sampling_step_2:
        r_eta = 1.0 - 0.5 * math.exp(-float(itr - args.r_sampling_step_1) / args.r_exp_alpha)
    else:
        r_eta = 1.0
    #eta 表示正向调度采样的概率值
    if itr < args.r_sampling_step_1:
        eta = 0.5
    elif itr < args.r_sampling_step_2:
        eta = 0.5 - (0.5 / (args.r_sampling_step_2 - args.r_sampling_step_1)) * (itr - args.r_sampling_step_1)
    else:
        eta = 0.0

    # 生成逆向调度采样的标志
    r_random_flip = np.random.random_sample(
        (args.batch_size, args.input_length - 1))#生成一个元素的值在 [0, 1) 之间的数组用于后续判断是否进行逆向调度采样。
    """
    该行代码的功能是根据生成的随机数数组 `r_random_flip` 和逆向调度采样的概率值 `r_eta`，生成一个布尔数组 `r_true_token`。
    具体来说，如果 `r_random_flip` 中的元素小于 `r_eta`，则对应的 `r_true_token` 元素为 `True`，否则为 `False`。
    """
    r_true_token = (r_random_flip < r_eta)
    #下面这段代码和上面的那个差不多，是正向调度算法
    random_flip = np.random.random_sample(
        (args.batch_size, args.total_length - args.input_length - 1))
    true_token = (random_flip < eta)
    #创建两个三维数组 ones 和 zeros，用于后续生成采样标志，这两个数组的形状由图像宽度、高度和通道数决定。
    ones = np.ones((args.img_width // args.patch_size,
                    args.img_width // args.patch_size,
                    args.patch_size ** 2 * args.img_channel))
    zeros = np.zeros((args.img_width // args.patch_size,
                      args.img_width // args.patch_size,
                      args.patch_size ** 2 * args.img_channel))

    real_input_flag = []
    for i in range(args.batch_size):
        for j in range(args.total_length - 2):
            if j < args.input_length - 1:
                if r_true_token[i, j]:
                    real_input_flag.append(ones)
                else:
                    real_input_flag.append(zeros)
            else:
                if true_token[i, j - (args.input_length - 1)]:
                    real_input_flag.append(ones)
                else:
                    real_input_flag.append(zeros)

    real_input_flag = np.array(real_input_flag)
    real_input_flag = np.reshape(real_input_flag,
                                 (args.batch_size,
                                  args.total_length - 2,
                                  args.img_width // args.patch_size,
                                  args.img_width // args.patch_size,
                                  args.patch_size ** 2 * args.img_channel))
    return real_input_flag

# 定义调度采样函数
def schedule_sampling(eta, itr):
    """
    根据当前迭代次数和给定的eta值计算调度采样的概率，并生成相应的采样标志。

    参数:
    eta (float): 当前的eta值
    itr (int): 当前迭代次数

    返回:
    eta (float): 更新后的eta值
    real_input_flag (np.ndarray): 采样标志数组
    """
    zeros = np.zeros((args.batch_size,
                      args.total_length - args.input_length - 1,
                      args.img_width // args.patch_size,
                      args.img_width // args.patch_size,
                      args.patch_size ** 2 * args.img_channel))
    if not args.scheduled_sampling:
        return 0.0, zeros

    if itr < args.sampling_stop_iter:
        eta -= args.sampling_changing_rate
    else:
        eta = 0.0
    random_flip = np.random.random_sample(
        (args.batch_size, args.total_length - args.input_length - 1))
    true_token = (random_flip < eta)
    ones = np.ones((args.img_width // args.patch_size,
                    args.img_width // args.patch_size,
                    args.patch_size ** 2 * args.img_channel))
    zeros = np.zeros((args.img_width // args.patch_size,
                      args.img_width // args.patch_size,
                      args.patch_size ** 2 * args.img_channel))
    real_input_flag = []
    for i in range(args.batch_size):
        for j in range(args.total_length - args.input_length - 1):
            if true_token[i, j]:
                real_input_flag.append(ones)
            else:
                real_input_flag.append(zeros)
    real_input_flag = np.array(real_input_flag)
    real_input_flag = np.reshape(real_input_flag,
                                 (args.batch_size,
                                  args.total_length - args.input_length - 1,
                                  args.img_width // args.patch_size,
                                  args.img_width // args.patch_size,
                                  args.patch_size ** 2 * args.img_channel))
    return eta, real_input_flag


In [11]:
from core.utils import preprocess
from core.data_provider import datasets_factory

def train_wrapper(model):
    """
    包装训练过程，包括加载预训练模型、数据加载、训练和测试。
    
    参数:
    model (Model): 训练模型实例
    """
    if args.pretrained_model:
        model.load(args.pretrained_model)
    # 加载数据，args.injection_action：是否使用某种特定的数据增强或特性注入
    train_input_handle, test_input_handle = datasets_factory.data_provider(
        args.dataset_name, args.train_data_paths, args.valid_data_paths, args.batch_size, args.img_width,
        seq_length=args.total_length, injection_action=args.injection_action, is_training=True)

    eta = args.sampling_start_value
    
    for itr in range(1, args.max_iterations + 1):
        # 检查数据是否用完
        if train_input_handle.no_batch_left():
            train_input_handle.begin(do_shuffle=True)  # 重新洗牌数据
        
        # 读取一个批次的训练数据
        ims = train_input_handle.get_batch()
        ims = preprocess.reshape_patch(ims, args.patch_size)
        
        # 采样策略（不影响数据本身）
        if args.reverse_scheduled_sampling == 1:
            real_input_flag = reserve_schedule_sampling_exp(itr)
        else:
            eta, real_input_flag = schedule_sampling(eta, itr)
        
        # 输出数据形状
        print(f"Iteration {itr}: ims shape = {ims.shape}")
        print(f"Sample values (first 3 elements): {ims.flatten()[:3]}")
        
        train_input_handle.next()  # 读取下一个批次


In [13]:
from core.models.model_factory import Model

# 创建模型实例
model = Model(args)
train_wrapper(model)


ValueError: Object arrays cannot be loaded when allow_pickle=False